# Imports

In [76]:
import pandas as pd
import yfinance as yf

# Dataset

In [77]:
daily_sentiment = pd.read_csv("../datasets/fng_data.csv")
btc_price_action = yf.Ticker("BTC-USD")
btc_price_action = btc_price_action.history("max")
btc_price_action.index = pd.to_datetime(btc_price_action.index).tz_localize(None)
btc_price_action.columns = map(str.lower, btc_price_action.columns)
del btc_price_action["dividends"]
del btc_price_action["stock splits"]
btc_price_action.head()

,open,high,low,close,volume
Date,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,21056800
2014-09-18,456.859985,456.859985,413.104004,424.440002,34483200
2014-09-19,424.102997,427.834991,384.532013,394.795990,37919700
2014-09-20,394.673004,423.295990,389.882996,408.903992,36863600
2014-09-21,408.084991,412.425995,393.181000,398.821014,26580100


Changing the date in btc_price_action into datetime

In [78]:
print(daily_sentiment['date'].dtype)  # This should ideally be 'object' indicating string

# Convert the column to datetime
daily_sentiment['date'] = pd.to_datetime(daily_sentiment['date'], dayfirst=True)

# Now, you can use .dt accessor
daily_sentiment['date'] = daily_sentiment['date'].dt.date

daily_sentiment.set_index('date', inplace=True)

# Drop the old index if it exists
if 'Unnamed: 0' in daily_sentiment.columns:
    daily_sentiment.drop(columns=['Unnamed: 0'], inplace=True)

# Display the DataFrame with 'date' as index and the old index dropped
display(daily_sentiment)

object


,fng_value,fng_classification
date,,
2024-04-06,75,Greed
2024-04-05,79,Extreme Greed
2024-04-04,70,Greed
2024-04-03,71,Greed
2024-04-02,79,Extreme Greed
...,...,...
2018-02-05,11,Extreme Fear
2018-02-04,24,Extreme Fear
2018-02-03,40,Fear


In [79]:
start_date = daily_sentiment.index.min()
end_date = daily_sentiment.index.max()

# Feature engineering

In [80]:
def stochastic_oscillator(df, k_length=5, k_smoothing=3, d_smoothing=3):
    df['low_min'] = df['low'].rolling(window=k_length).min()
    df['high_max'] = df['high'].rolling(window=k_length).max()
    df['stoch_k'] = ((df['close'] - df['low_min']) / (df['high_max'] - df['low_min'])) * 100
    df['stoch_k_smooth'] = df['stoch_k'].rolling(window=k_smoothing).mean()
    df['stoch_d'] = df['stoch_k_smooth'].rolling(window=d_smoothing).mean()
    df.drop(columns=['low_min', 'high_max'], inplace=True)
    return df

# Calculate Moving Averages
def calculate_ma(df, ma_periods=[5, 13]):
    for period in ma_periods:
        df[f'ma_{period}'] = df['close'].rolling(window=period).mean()
    return df

# Apply the stochastic oscillator function to your DataFrame
btc_price_action = stochastic_oscillator(btc_price_action)

# Apply the moving averages calculation to your DataFrame
btc_price_action = calculate_ma(btc_price_action)

# Display the DataFrame with Stochastic Oscillator and Moving Averages values
display(btc_price_action)

,open,high,low,close,volume,stoch_k,stoch_k_smooth,stoch_d,ma_5,ma_13
Date,,,,,,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,21056800,NaN,NaN,NaN,NaN,NaN
2014-09-18,456.859985,456.859985,413.104004,424.440002,34483200,NaN,NaN,NaN,NaN,NaN
2014-09-19,424.102997,427.834991,384.532013,394.795990,37919700,NaN,NaN,NaN,NaN,NaN
2014-09-20,394.673004,423.295990,389.882996,408.903992,36863600,NaN,NaN,NaN,NaN,NaN
2014-09-21,408.084991,412.425995,393.181000,398.821014,26580100,17.083525,NaN,NaN,416.859003,NaN
...,...,...,...,...,...,...,...,...,...,...
2024-05-03,59122.300781,63320.503906,58848.312500,62889.835938,33172023048,77.743159,43.369954,35.068729,60949.051562,63227.934796
2024-05-04,62891.031250,64494.957031,62599.351562,63891.472656,20620477992,90.036138,66.432603,45.391938,60959.121875,63148.306190
2024-05-05,63892.453125,64610.890625,62955.304688,64031.132812,18296164805,92.803044,86.860780,65.554446,61637.977344,62932.417969


In [81]:
btc_price_action = btc_price_action.loc[start_date:end_date]

In [82]:
btc_price_action = btc_price_action.merge(daily_sentiment, left_index=True, right_index=True, how='left')

In [83]:
btc_price_action.fillna(method='ffill', inplace=True)

C:\Users\Jehoiada Wong\AppData\Local\Temp\ipykernel_18040\710988571.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  btc_price_action.fillna(method='ffill', inplace=True)


In [84]:
btc_price_action.head(3)

,open,high,low,close,volume,stoch_k,stoch_k_smooth,stoch_d,ma_5,ma_13,fng_value,fng_classification
Date,,,,,,,,,,,,
2018-02-01,10237.299805,10288.799805,8812.280273,9170.540039,9959400448,11.098439,11.401036,27.762554,10516.127930,11085.433969,30.0,Fear
2018-02-02,9142.280273,9142.280273,7796.490234,8830.750000,12726899712,25.355039,18.686779,18.442888,9925.017969,10772.476262,15.0,Extreme Fear
2018-02-03,8852.120117,9430.750000,8251.629883,9174.910156,7263790080,39.263281,25.238920,18.442245,9500.719922,10585.923227,40.0,Fear


In [85]:
btc_price_action['close t-3'] = btc_price_action['close'].shift(3)
btc_price_action['close t-5'] = btc_price_action['close'].shift(5)
btc_price_action['close t-7'] = btc_price_action['close'].shift(7)
btc_price_action['close t-12'] = btc_price_action['close'].shift(12)
btc_price_action['close t-30'] = btc_price_action['close'].shift(30)
btc_price_action['close t+1'] = btc_price_action['close'].shift(-1)
btc_price_action['close t+3'] = btc_price_action['close'].shift(-3)
btc_price_action['close t+5'] = btc_price_action['close'].shift(-5)
btc_price_action['close t+7'] = btc_price_action['close'].shift(-7)
btc_price_action['close t+12'] = btc_price_action['close'].shift(-12)
btc_price_action['close t+30'] = btc_price_action['close'].shift(-30)

In [86]:
btc_price_action.columns.values

array(['open', 'high', 'low', 'close', 'volume', 'stoch_k',
       'stoch_k_smooth', 'stoch_d', 'ma_5', 'ma_13', ' fng_value',
       'fng_classification', 'close t-3', 'close t-5', 'close t-7',
       'close t-12', 'close t-30', 'close t+1', 'close t+3', 'close t+5',
       'close t+7', 'close t+12', 'close t+30'], dtype=object)

In [87]:
btc_price_action = btc_price_action[['open', 'high', 'low', 'close', 'close t-3', 'close t-5', 'close t-7','close t-12', 'close t-30', 'close t+3', 'close t+5', 'close t+7'
                                   ,'close t+12', 'close t+30', 'volume', 'stoch_k',
                                   'stoch_k_smooth', 'stoch_d', 'ma_5', 'ma_13', ' fng_value',
                                   'fng_classification']]
btc_price_action.head(10)

,open,high,low,close,close t-3,close t-5,close t-7,close t-12,close t-30,close t+3,...,close t+12,close t+30,volume,stoch_k,stoch_k_smooth,stoch_d,ma_5,ma_13,fng_value,fng_classification
Date,,,,,,,,,,,,,,,,,,,,,
2018-02-01,10237.299805,10288.799805,8812.280273,9170.540039,NaN,NaN,NaN,NaN,NaN,8277.009766,...,8598.309570,11489.700195,9959400448,11.098439,11.401036,27.762554,10516.127930,11085.433969,30.0,Fear
2018-02-02,9142.280273,9142.280273,7796.490234,8830.750000,NaN,NaN,NaN,NaN,NaN,6955.270020,...,9494.629883,11512.599609,12726899712,25.355039,18.686779,18.442888,9925.017969,10772.476262,15.0,Extreme Fear
2018-02-03,8852.120117,9430.750000,8251.629883,9174.910156,NaN,NaN,NaN,NaN,NaN,7754.000000,...,10166.400391,11573.299805,7263790080,39.263281,25.238920,18.442245,9500.719922,10585.923227,40.0,Fear
2018-02-04,9175.700195,9334.870117,8031.220215,8277.009766,9170.540039,NaN,NaN,NaN,NaN,7621.299805,...,10233.900391,10779.900391,7073549824,18.587977,27.735432,23.887044,9134.861914,10381.739333,24.0,Extreme Fear
2018-02-05,8270.540039,8364.839844,6756.680176,6955.270020,8830.750000,NaN,NaN,NaN,NaN,8265.589844,...,11112.700195,9965.570312,9285289984,5.622399,21.157886,24.710746,8481.695996,10080.729304,11.0,Extreme Fear
2018-02-06,7051.750000,7850.700195,6048.259766,7754.000000,9174.910156,9170.540039,NaN,NaN,NaN,8736.980469,...,10551.799805,9395.009766,13999800320,50.428534,24.879636,24.590985,8198.387988,9803.390813,8.0,Extreme Fear
2018-02-07,7755.490234,8509.110352,7236.790039,7621.299805,8277.009766,8830.750000,NaN,NaN,NaN,8621.900391,...,11225.299805,9337.549805,9169280000,46.505383,34.185438,26.740987,7956.497949,9523.536922,36.0,Fear
2018-02-08,7637.859863,8558.769531,7637.859863,8265.589844,6955.270020,9174.910156,9170.540039,NaN,NaN,8129.970215,...,11403.700195,8866.000000,9346750464,67.465560,54.799826,37.954967,7774.633887,9300.013033,30.0,Fear
2018-02-09,8271.839844,8736.980469,7884.709961,8736.980469,7754.000000,8277.009766,8830.750000,NaN,NaN,8926.570312,...,10690.400391,9578.629883,6784820224,100.000000,71.323648,53.436304,7866.628027,9092.034593,44.0,Fear


In [88]:
btc_price_action.to_csv("../datasets/final_data.csv")

In [89]:
df = pd.read_csv("../datasets/final_data.csv")

In [90]:
df

,Date,open,high,low,close,close t-3,close t-5,close t-7,close t-12,close t-30,...,close t+12,close t+30,volume,stoch_k,stoch_k_smooth,stoch_d,ma_5,ma_13,fng_value,fng_classification
0,2018-02-01,10237.299805,10288.799805,8812.280273,9170.540039,NaN,NaN,NaN,NaN,NaN,...,8598.309570,11489.700195,9959400448,11.098439,11.401036,27.762554,10516.127930,11085.433969,30.0,Fear
1,2018-02-02,9142.280273,9142.280273,7796.490234,8830.750000,NaN,NaN,NaN,NaN,NaN,...,9494.629883,11512.599609,12726899712,25.355039,18.686779,18.442888,9925.017969,10772.476262,15.0,Extreme Fear
2,2018-02-03,8852.120117,9430.750000,8251.629883,9174.910156,NaN,NaN,NaN,NaN,NaN,...,10166.400391,11573.299805,7263790080,39.263281,25.238920,18.442245,9500.719922,10585.923227,40.0,Fear
3,2018-02-04,9175.700195,9334.870117,8031.220215,8277.009766,9170.540039,NaN,NaN,NaN,NaN,...,10233.900391,10779.900391,7073549824,18.587977,27.735432,23.887044,9134.861914,10381.739333,24.0,Extreme Fear
4,2018-02-05,8270.540039,8364.839844,6756.680176,6955.270020,8830.750000,NaN,NaN,NaN,NaN,...,11112.700195,9965.570312,9285289984,5.622399,21.157886,24.710746,8481.695996,10080.729304,11.0,Extreme Fear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2252,2024-04-02,69705.023438,69708.382812,64586.593750,65446.972656,69645.304688,70744.953125,69987.835938,65491.390625,63167.371094,...,NaN,NaN,50705240709,12.669050,49.072621,56.774907,69204.180469,68210.336538,79.0,Extreme Greed
2253,2024-04-03,65446.671875,66914.320312,64559.898438,65980.812500,71333.648438,69892.828125,69455.343750,63778.761719,68330.414062,...,NaN,NaN,34488018367,20.840987,26.612041,44.373701,68421.777344,68247.984375,71.0,Greed
2254,2024-04-04,65975.695312,69291.257812,65113.796875,68508.843750,69702.148438,69645.304688,70744.953125,64062.203125,63801.199219,...,NaN,NaN,34439527442,57.920405,30.476814,35.387159,68194.485156,68611.836839,70.0,Greed
2255,2024-04-05,68515.757812,68725.757812,66011.476562,67837.640625,65446.972656,71333.648438,69892.828125,67234.171875,66106.804688,...,NaN,NaN,33748230056,48.328632,42.363341,33.150732,67495.283594,68902.255108,79.0,Extreme Greed
